In [ ]:
import json
import os
import _jsonnet
import os

In [ ]:
from seq2struct.commands.infer import Inferer
from seq2struct.datasets.spider import SpiderItem
from seq2struct.utils import registry

In [ ]:
import torch

In [ ]:
exp_config = json.loads(
    _jsonnet.evaluate_file(
        "experiments/spider-configs/gap-run.jsonnet"))


In [ ]:
model_config_path = exp_config["model_config"]
model_config_args = exp_config.get("model_config_args")

In [ ]:
infer_config = json.loads(
    _jsonnet.evaluate_file(
        model_config_path, 
        tla_codes={'args': json.dumps(model_config_args)}))

In [ ]:
infer_config["model"]["encoder_preproc"]["db_path"] = "data/sqlite_files/"

In [ ]:
inferer = Inferer(infer_config)

In [ ]:
model_dir = exp_config["logdir"] + "/bs=12,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1"
checkpoint_step = exp_config["eval_steps"][0]

In [ ]:
model = inferer.load_model(model_dir, checkpoint_step)

In [ ]:
from seq2struct.datasets.spider_lib.preprocess.get_tables import dump_db_json_schema
from seq2struct.datasets.spider import load_tables_from_schema_dict

In [ ]:
db_id = "singer"

In [ ]:
my_schema = dump_db_json_schema("data/sqlite_files/{db_id}/{db_id}.sqlite".format(db_id=db_id), db_id)

In [ ]:
from seq2struct.utils.api_utils import refine_schema_names

In [ ]:
my_schema

In [ ]:
# If you want to change your schema name, then run this; Otherwise you can skip this.
refine_schema_names(my_schema)

In [ ]:
schema, eval_foreign_key_maps = load_tables_from_schema_dict(my_schema)

In [ ]:
schema.keys()

In [ ]:
dataset = registry.construct('dataset_infer', {
   "name": "spider", "schemas": schema, "eval_foreign_key_maps": eval_foreign_key_maps, 
    "db_path": "data/sqlite_files/"
})

In [ ]:
for _, schema in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema)

In [ ]:
spider_schema = dataset.schemas[db_id]

In [ ]:
def infer(question):
    data_item = SpiderItem(
            text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
            code=None,
            schema=spider_schema,
            orig_schema=spider_schema.orig,
            orig={"question": question}
        )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        output = inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)
    return output[0]["inferred_code"]

In [ ]:
code = infer("How many singers are there?")
print(code)